In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import random
import time
import tensorflow as tf

In [ ]:
n_train = 2320
n_valid = 1048

In [ ]:
IMG_SIZE = 160

def load(dir, n):
    x_data, y_data = [], []
    for i in range(n):
        image = cv2.imread(dir + "/" + str(i + 1) + ".png")
        r = open(dir + "/" + str(i + 1) + ".txt", "r")
        label = int(r.readline())
        image = tf.cast(image, tf.float32)
        image = (image/127.5) - 1
        x_data.append(image)
        y_data.append(label)
    return x_data, y_data

X_train, Y_train = load("/home/dima/PycharmProjects/Practics/Dataset/train", n_train)
X_valid, Y_valid = load("/home/dima/PycharmProjects/Practics/Dataset/valid", n_valid)
Y_train = tf.cast(Y_train, tf.int32)
Y_valid = tf.cast(Y_valid, tf.int32)
Y_train -= 1
Y_valid -= 1
print(Y_train)
X_train_norm = tf.norm(X_train)
X_valid_norm = tf.norm(X_valid)

#X_train = X_train / X_train_norm
#X_valid = X_valid / X_valid_norm


In [ ]:
train = tf.data.Dataset.from_tensor_slices( (X_train, Y_train) )
valid = tf.data.Dataset.from_tensor_slices( (X_valid, Y_valid) )

In [ ]:
print(train)
print(valid)

In [ ]:
BATCH_SIZE = 29
SHUFFLE_BUFFER_SIZE = 1000

In [ ]:
train_batches = train.batch(BATCH_SIZE)
validation_batches = valid.batch(BATCH_SIZE)
print(train_batches, validation_batches)

In [ ]:
for image_batch, label_batch in train_batches.take(1):
    pass

image_batch.shape

In [ ]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)


base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


In [ ]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)


In [ ]:
base_model.trainable = False
base_model.summary()


In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)


In [ ]:
import tensorflow.keras as keras

prediction_layer = keras.layers.Dense(4, activation = 'softmax')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)


In [ ]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])
model.summary()

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
print(len(model.trainable_variables))
for image_batch, label_batch in validation_batches:
    print(label_batch)

In [ ]:
validation_steps = 37
initial_epochs = 20
model.evaluate(validation_batches, steps = validation_steps)
predictions = model.predict(validation_batches, steps = validation_steps, verbose = 1)
print(predictions)

In [ ]:

history = model.fit(train_batches,
                    epochs=initial_epochs, validation_data=validation_batches)


In [ ]:
predictions = model.predict(validation_batches, steps = validation_steps, verbose = 1)
print(predictions)
for p in predictions:
    print(p)

In [ ]:
model.evaluate(train_batches, steps = 100)

In [ ]:
base_model.trainable = True

print("Number of layers in the base model: ", len(base_model.layers))

fine_tune_at = 100

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])

model.summary()


In [ ]:
len(model.trainable_variables)

In [ ]:
fine_tune_epochs = 50
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_batches,
                         epochs=total_epochs,
                         initial_epoch =  history.epoch[-1],
                         validation_data=validation_batches)


In [ ]:
model.evaluate(train_batches, steps = 100)

In [ ]:
predictions = model.predict(validation_batches, steps = validation_steps, verbose = 1)
print(predictions)
for p in predictions:
    print(p)